In [9]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service as FirefoxService
from unidecode import unidecode
from pathlib import Path
from datetime import datetime, timedelta
from lingua import Language, LanguageDetectorBuilder
import pandas as pd
import json
import time

In [10]:
# url = "https://www.google.com/travel/hotels/entity/ChYIqtL21OvSv65QGgovbS8wdnB3cTRzEAE/reviews?utm_campaign=sharing&utm_medium=link&utm_source=htls&ts=CAEaIAoCGgASGhIUCgcI6A8QAxgLEgcI6A8QAxgMGAEyAhAAKgkKBToDTUdBGgA&ved=0CAAQ5JsGahcKEwiY1evEleyEAxUAAAAAHQAAAAAQAw"
# url = "https://www.google.com/travel/hotels/entity/ChYIqtL21OvSv65QGgovbS8wdnB3cTRzEAE/reviews?utm_campaign=sharing&utm_medium=link&utm_source=htls"
url = "https://www.google.com/search?sa=X&sca_esv=631c077ebeab8f5b&tbm=lcl&sxsrf=ACQVn0-Vrn-6SBs84xf8oDcggXXtf0T_AQ:1711639111610&q=Le+Carre+d'As+Avis&rflfq=1&num=20&stick=H4sIAAAAAAAAAONgkxIxNLG0tLSwtDQzNjM3sjQ2MbUwstzAyPiKUcgnVcE5sagoVSFF3bFYwbEss3gRKxZBANr-AYtFAAAA&rldimm=14999899636729345829&hl=fr-FR&ved=2ahUKEwj8hezJoJeFAxVyUqQEHbrzDs4Q9fQKegQIOxAF&biw=1920&bih=927&dpr=1#lkt=LocalPoiReviews"

In [11]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('disable-translate')
chrome_options.add_argument('--lang=es')

In [12]:
# driver = webdriver.Chrome(options=chrome_options)
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument('--disable-gpu')
firefox_options.add_argument('--ignore-certificate-errors')
firefox_options.add_argument('--incognito')
firefox_options.set_preference('intl.accept_languages', 'en-US, en')
driver = webdriver.Firefox(service=FirefoxService(
                    GeckoDriverManager().install()), options=firefox_options)

[WDM] - Getting latest mozilla release info for v0.34.0
[WDM] - Trying to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.34.0/geckodriver-v0.34.0-win64.zip
[WDM] - Driver has been saved in cache [C:\Users\HP\.wdm\drivers\geckodriver\win64\v0.34.0]


In [13]:
driver.get(url)

In [14]:
try:
    driver.execute_script(f"window.scrollTo(0, 500);")
    order_dropdown = driver.find_element(By.XPATH, "//div[@jsname='wQNmvb']")
    driver.execute_script("arguments[0].click();", order_dropdown)
    time.sleep(2)
    order_item = driver.find_elements(By.XPATH, "//div[@jsname='V68bde']/div[@jsname='wQNmvb']")[1]
    driver.execute_script("arguments[0].click();", order_item)
    time.sleep(1)
except Exception as e:
    print("pass review order ...")
    print(e)
pass

pass review order ...
Message: Unable to locate element: //div[@jsname='wQNmvb']
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16



In [15]:
soupe = BeautifulSoup(driver.page_source, 'lxml')

In [16]:
container = soupe.find('div', {'jsname':'SvNErb'})

In [17]:
cards = container.find_all('div', {'class':'Svr5cf bKhjM'})
len(cards)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
# author =  cards[0].find('a', {'class':'DHIhE QB2Jof'}).text
# cards[1].find('span', {'class': 'iUtr1 CQYfx'}).text.strip().lower()[4:] 
# cards[0].find('div', {'class':'K7oBsc'}).find('span').text.replace('(Traducido por Google) ', '').replace('\xa0... Ver más', '').strip()

In [ ]:
try:
    driver.find_element(By.XPATH, "//button[@jsname='tWT92d']").click()
except:
    pass
time.sleep(10)

In [ ]:
def detect(text:str) -> str:
    if text:
        lang_code = {
        'Language.ENGLISH':'en',
        'Language.GERMAN':'de',
        'Language.SPANISH':'es',
        'Language.FRENCH':'fr',
    }

    languages = [Language.ENGLISH, Language.FRENCH, Language.GERMAN, Language.SPANISH]
    detector = LanguageDetectorBuilder.from_languages(*languages).build()
    try:
        return lang_code[f"{detector.detect_language_of(text)}"]
    except:
        return ''

In [ ]:
def formate_date(raw_date):
    split_date = raw_date.split(' ')
    today = datetime.now()
    language = detect(raw_date)
    if 'mes' in raw_date:
        language = 'es'
    print(language)
    match language:
        case "fr":

            if split_date[4] == 'jour':
                return datetime.strftime(today + timedelta(days=-1), '%d/%m/%Y')
            elif split_date[4] == 'jours':
                return datetime.strftime(today + timedelta(days=-(int(split_date[3]))), '%d/%m/%Y')
            if split_date[4] == 'semaine':
                return datetime.strftime(today + timedelta(days=-7), '%d/%m/%Y')
            elif split_date[4] == 'semaines':
                return datetime.strftime(today + timedelta(days=-7*int(split_date[3])), '%d/%m/%Y')
            elif split_date[4] == 'mois':
                if split_date[3] == 'un':
                    return datetime.strftime(today + timedelta(days=-31), '%d/%m/%Y')
                else:
                    return datetime.strftime(today + timedelta(days=-31*int(split_date[3])), '%d/%m/%Y')
            elif split_date[4] == 'an':
                return datetime.strftime(today + timedelta(days=-365), '%d/%m/%Y')
            elif split_date[4] == 'ans':
                return datetime.strftime(today + timedelta(days=-(int(split_date[3])*365)), '%d/%m/%Y')
            else:
                return datetime.strftime(today, '%d/%m/%Y')

        case"en":
            if split_date[1] == 'days':
                return datetime.strftime(today + timedelta(days=-(int(split_date[0]))), '%d/%m/%Y')
            if split_date[1] == 'week':
                return datetime.strftime(today + timedelta(days=-7), '%d/%m/%Y')
            elif split_date[1] == 'weeks':
                return datetime.strftime(today + timedelta(days=-7*int(split_date[0])), '%d/%m/%Y')
            elif split_date[1] == 'months' or split_date[1] == 'month':
                if split_date[0] == 'a':
                    return datetime.strftime(today + timedelta(days=-31), '%d/%m/%Y')
                else:
                    return datetime.strftime(today + timedelta(days=-31*int(split_date[0])), '%d/%m/%Y')
            elif split_date[1] == 'year':
                return datetime.strftime(today + timedelta(days=-365), '%d/%m/%Y')
            elif split_date[1] == 'years':
                return datetime.strftime(today + timedelta(days=-(int(split_date[0])*365)), '%d/%m/%Y')
            else:
                return datetime.strftime(today, '%d/%m/%Y')
        case "es":
            if split_date[1] == 'día':
                return datetime.strftime(today + timedelta(days=-1), '%d/%m/%Y')
            if split_date[1] == 'días':
                return datetime.strftime(today + timedelta(days=-(int(split_date[0]))), '%d/%m/%Y')
            if split_date[1] == 'semana':
                return datetime.strftime(today + timedelta(days=-7), '%d/%m/%Y')
            elif split_date[1] == 'semanas':
                return datetime.strftime(today + timedelta(days=-7*int(split_date[0])), '%d/%m/%Y')
            elif split_date[1] == 'mes' or split_date[1] == 'meses':
                if split_date[0] == 'un':
                    return datetime.strftime(today + timedelta(days=-31), '%d/%m/%Y')
                else:
                    return datetime.strftime(today + timedelta(days=-31*int(split_date[0])), '%d/%m/%Y')
            elif split_date[1] == 'año':
                return datetime.strftime(today + timedelta(days=-365), '%d/%m/%Y')
            elif split_date[1] == 'años':
                return datetime.strftime(today + timedelta(days=-(int(split_date[0])*365)), '%d/%m/%Y')
            else:
                return datetime.strftime(today, '%d/%m/%Y')

In [ ]:
import random
from urllib.parse import urlparse, parse_qs


def extract() -> list:
    print('extraction..')

    time.sleep(3)

    # try:

    try:
        accept_btn = driver.find_element(
            By.XPATH, "//span[contains(text(), 'Tout accepter') or contains(text(), 'Accept all')]")
        driver.execute_script("arguments[0].click();", accept_btn)
        time.sleep(random.randint(2, 5))
    except:
        pass

    time.sleep(5)

    try:
        driver.find_element(
            By.XPATH, "//div[@role='listbox' and @aria-label='Menu déroulant pour filtrer les avis']").click()
        time.sleep(random.uniform(.5, 2.5))
        driver.find_element(
            By.XPATH, "//div[@role='option' and @data-value='2' and @aria-label='Plus récents']").click()
        time.sleep(random.uniform(.2, 2))
    except:
        pass

    reviews = []

    page = driver.page_source

    soupe = BeautifulSoup(page, 'lxml')
    container = soupe.find('div', {'jsname':'SvNErb'})
    cards = container.find_all('div', {'class':'Svr5cf bKhjM'})

    for card in cards:
        # author = card.find('span', {'class': 'k5TI0'}).find('a').text.strip() if card.find('span', {'class': 'k5TI0'}) and card.find('span', {'class': 'k5TI0'}).find('a') else ""
        author = card.find('a', {'class':'DHIhE QB2Jof'}).text.strip() if card.find('a', {'class':'DHIhE QB2Jof'}) else ""
        # comment = card.find('div', {'class': 'K7oBsc'}).find('span').text.strip().replace(
        #     " En savoir plus", "").replace('(Traducido por Google)', '') \
        #     .replace('(Traduit par Google)', '').repalce('(Taduce by Google)', '') if card.find('div', {'class': 'K7oBsc'}) else ""
        try:
            comment = card.find('div', {'class':'K7oBsc'}).find('span').text.replace('(Traducido por Google) ', '').replace('\xa0... Ver más', '').replace(" En savoir plus", "") \
            .replace('(Traduit par Google)', '').replace('(Traduce by Google)', '').lower().split('(original)')[-1] if card.find('div', {'class':'K7oBsc'}) else ""
        except:
            comment = ""
        rating = card.find('div', {'class': 'GDWaad'}).text.strip().split(
            '/')[0] if card.find('div', {'class': 'GDWaad'}) else "0"
        try:
            lang = detect(comment)
        except:
            lang = 'en'
        date_raw = card.find('span', {'class': 'iUtr1 CQYfx'}).text.lower()
        match(detect(date_raw)):
            case 'es':
                date_raw = date_raw.replace('hace ', '').replace('\xa0','').strip() if card.find('span', {'class': 'iUtr1 CQYfx'}) else ""
            case _:
                date_raw = date_raw.text.lower().replace('\xa0','').strip() if card.find('span', {'class': 'iUtr1 CQYfx'}) else ""

        print(date_raw)
        date_review = formate_date(date_raw) if date_raw else "01/01/1999"
        if date_review:
            if (author or comment or rating != "0") and datetime.strptime(date_review, '%d/%m/%Y') > datetime.now() - timedelta(days=365):

                reviews.append({
                    'rating': rating,
                    'author': author,
                    'date_review': date_review,
                    'comment': comment,
                    'language': lang,
                    'source': urlparse(driver.current_url).netloc.split('.')[1],
                    'date_visit': date_review,
                    'novisitday': "1"
                })

                print({
                    'rating': rating,
                    'author': author,
                    'date_review': date_review,
                    'comment': comment,
                    'language': lang,
                    'source': urlparse(driver.current_url).netloc.split('.')[1],
                    'date_visit': date_review,
                    'novisitday': "1"
                })

    print(reviews)
    return reviews
    # except Exception as e:
    #     print(e)
    #     pass


In [ ]:
extract()

extraction..
18horas en  google
es
{'rating': '4', 'author': 'MissMinimish', 'date_review': '20/03/2024', 'comment': 'no es la primera vez aquí.las habitaciones están empezando a verse y sentirse un poco cansadas. a las alfombras les vendría bien una buena limpieza.', 'language': 'es', 'source': 'google', 'date_visit': '20/03/2024', 'novisitday': '1'}
un día en  google
es
{'rating': '5', 'author': 'AJ JA', 'date_review': '19/03/2024', 'comment': 'otra estancia maravillosa en la línea alta estándar.ubicación perfecta para explorar la ciudad.personal de recepción cortés y servicial.', 'language': 'es', 'source': 'google', 'date_visit': '19/03/2024', 'novisitday': '1'}
2días en  google
es
{'rating': '5', 'author': 'Sonjala Williams', 'date_review': '20/03/2024', 'comment': '', 'language': '', 'source': 'google', 'date_visit': '20/03/2024', 'novisitday': '1'}
3días en  google
es
{'rating': '1', 'author': 'C Halet', 'date_review': '20/03/2024', 'comment': 'horrible customer service.', 'lang

[{'rating': '4',
  'author': 'MissMinimish',
  'date_review': '20/03/2024',
  'comment': 'no es la primera vez aquí.las habitaciones están empezando a verse y sentirse un poco cansadas. a las alfombras les vendría bien una buena limpieza.',
  'language': 'es',
  'source': 'google',
  'date_visit': '20/03/2024',
  'novisitday': '1'},
 {'rating': '5',
  'author': 'AJ JA',
  'date_review': '19/03/2024',
  'comment': 'otra estancia maravillosa en la línea alta estándar.ubicación perfecta para explorar la ciudad.personal de recepción cortés y servicial.',
  'language': 'es',
  'source': 'google',
  'date_visit': '19/03/2024',
  'novisitday': '1'},
 {'rating': '5',
  'author': 'Sonjala Williams',
  'date_review': '20/03/2024',
  'comment': '',
  'language': '',
  'source': 'google',
  'date_visit': '20/03/2024',
  'novisitday': '1'},
 {'rating': '1',
  'author': 'C Halet',
  'date_review': '20/03/2024',
  'comment': 'horrible customer service.',
  'language': 'en',
  'source': 'google',
  'd

In [ ]:
all_data = []

In [ ]:
def save_data(data_source:list) -> None:
    global all_data
    new_data = []
    df = pd.concat([pd.DataFrame(all_data), pd.DataFrame(data_source)])
    df.drop_duplicates(subset=['rating','author','date_review','comment','language','source','date_visit','novisitday'], inplace=True)
    for i in range(len(df)):
        new_data.append(df.iloc[i].to_dict())
    all_data = new_data

In [ ]:
def load_reviews():
    def get_last_review_date():
        page = driver.page_source
        soupe = BeautifulSoup(page, 'lxml')
        last_review_cards = soupe.find_all('div', {'jsname': "Pa5DKe"})[-1]
        date_raw = last_review_cards.find('span', {'class': 'iUtr1'}).text.strip(
        ) if last_review_cards.find('span', {'class': 'iUtr1'}) else ""
        comment = last_review_cards.find('div', {'class': 'K7oBsc'}).find('span').text.strip()\
        .replace(" En savoir plus", "") if last_review_cards.find('div', {'class': 'K7oBsc'}) else ""
        try:
            lang = detect(comment)
        except:
            lang = 'en'

        return formate_date(date_raw, lang) if date_raw else "01/01/1999"

    try:
        driver.find_element(
            By.XPATH, "//div[@aria-label='Avis' and @aria-controls='reviews' and @role='tab']").click()
        time.sleep(2)
    except:
        pass

    results = int(''.join([x for x in driver.find_element(
        By.CSS_SELECTOR, '#reviews > c-wiz > c-wiz > div > div > div > div > div.ChBWlb.TjtFVc > div.pDLIp > div > div.zhMoVd.nNUNpc > div.UkIqCb > div > span').text if x.isdigit()]))

    index = 0
    for i in range(results//6):
        # if not check_date(get_last_review_date()):
        #     break
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        if index == 10:
            for k in range(2):
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
            data = extract()
            save_data(data)
            index = 0
        index += 1

In [ ]:
load_reviews()

extraction..
17horas en  google
es
20/03/2024
{'rating': '4', 'author': 'MissMinimish', 'date_review': '20/03/2024', 'comment': 'no es la primera vez aquí.las habitaciones están empezando a verse y sentirse un poco cansadas. a las alfombras les vendría bien una buena limpieza.', 'language': 'es', 'source': 'google', 'date_visit': '20/03/2024', 'novisitday': '1'}
un día en  google
es
19/03/2024
{'rating': '5', 'author': 'AJ JA', 'date_review': '19/03/2024', 'comment': 'otra estancia maravillosa en la línea alta estándar.ubicación perfecta para explorar la ciudad.personal de recepción cortés y servicial.', 'language': 'es', 'source': 'google', 'date_visit': '19/03/2024', 'novisitday': '1'}
2días en  google
es
20/03/2024
{'rating': '5', 'author': 'Sonjala Williams', 'date_review': '20/03/2024', 'comment': '', 'language': '', 'source': 'google', 'date_visit': '20/03/2024', 'novisitday': '1'}
3días en  google
es
20/03/2024
{'rating': '1', 'author': 'C Halet', 'date_review': '20/03/2024', 'c

In [ ]:
(datetime.now() - timedelta(days=365)).strftime('%d-%m-%Y')

'14-03-2023'